In [24]:
import numpy as np
import seaborn as sn 
import matplotlib.pyplot as plt 
import pandas as pd
import string
from nltk.corpus import stopwords
sms=pd.read_csv(r"C:\Users\elham\Desktop\excel2\spam.csv",encoding='latin-1')

sms.head(2)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN


In [25]:
sms.dropna(how='any',inplace=True,axis=1)
sms.columns=['label','message']
sms.head(2)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [26]:
sms['label'].unique()
sms['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [27]:
sms['labelnm']=sms.label.map({'ham':0,'spam':1})
sms['msg_length']=sms.message.apply(len)
sms['msg_lower']=sms.message.apply(str.lower)
sms.head(1)

,label,message,labelnm,msg_length,msg_lower
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,"go until jurong point, crazy.. available only ..."


In [28]:
def text_process(mess):
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])
    

In [29]:
sms['clean_msg']=sms.message.apply(text_process)


In [30]:
y=sms.labelnm
x=sms.msg_lower
print(x.shape,y.shape)




(5572,) (5572,)


In [32]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)
print(x_train.shape)

(4179,)


In [38]:
x_train.head()

710     height of confidence: all the aeronautics prof...
3740                                        2/2 146tf150p
2711    wen ur lovable bcums angry wid u, dnt take it ...
3155                    long time. you remember me today.
3748    dear voucher holder 2 claim your 1st class air...
Name: msg_lower, dtype: object

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vect = CountVectorizer()

# Fit and transform the cleaned messages
X_train_dtm = vect.fit_transform(x_train)
X_test_dtm=vect.transform(x_test)
# Optionally, view the feature names
feature_names = vect.get_feature_names_out()
feature_names # Show first 10 features

array(['00', '000', '000pes', ..., 'ûïharry', 'ûò', 'ûówell'],
      dtype=object)

In [59]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='liblinear')
 # train the model using X_train_dtm
# Measure training time using %time (only works in Jupyter Notebooks)
%time model.fit(X_train_dtm, y_train)

CPU times: total: 0 ns
Wall time: 100 ms


LogisticRegression(solver='liblinear')

In [64]:

y_pred_class = model.predict(X_test_dtm)

y_pred_class.shape

(1393,)

In [48]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = model.predict_proba(X_test_dtm)
y_pred_prob

array([[9.94621785e-01, 5.37821462e-03],
       [9.61545850e-01, 3.84541501e-02],
       [9.95989125e-01, 4.01087518e-03],
       ...,
       [9.92512855e-01, 7.48714543e-03],
       [9.99583136e-01, 4.16863921e-04],
       [9.20909627e-01, 7.90903730e-02]])

In [49]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = model.predict_proba(X_train_dtm)[:, 1]
y_pred_prob

array([0.00537821, 0.03845415, 0.00401088, ..., 0.00748715, 0.00041686,
       0.07909037])

In [65]:
y_test.shape

(1393,)

In [66]:
import sklearn.metrics as metrics
# calculate accuracy of class predictions
print("=======Accuracy Score===========")
print(metrics.accuracy_score(y_test, y_pred_class))



=======Accuracy Score===========
0.9877961234745154


In [67]:
# print the confusion matrix
print("=======Confision Matrix===========")
print(metrics.confusion_matrix(y_test, y_pred_class))


=======Confision Matrix===========
[[1211    2]
 [  15  165]]


In [68]:

# calculate AUC
print("=======ROC AUC Score===========")
print(metrics.roc_auc_score(y_test, y_pred_class))

=======ROC AUC Score===========
0.9575089310250067


In [61]:
print(X_train_dtm.shape)  # Shape of training data
print(X_test_dtm.shape)   # Shape of test data
print(len(y_train))       # Number of training labels
print(len(y_test)) 

(4179, 7496)
(1393, 7496)
4179
1393


In [62]:
print(y_pred_class.shape) 


(1393,)
